# Model Building

In [ ]:
# Import libraries


In [ ]:
# Confirm access to feature store and feature groups


In [ ]:
# Split data into training/testing/validation/production sets

In [ ]:
# Perform Standardization scaling for regression (ONLY ON TRAINING DATA)
